#### kitt_net neural network training

In [1]:
from kitt_net import FeedForwardNet
from shelve import open as open_shelve
from sklearn.metrics import confusion_matrix, accuracy_score
from matplotlib import pyplot as plt
%matplotlib qt
import numpy as np
np.set_printoptions(threshold=np.nan)

In [2]:
net = FeedForwardNet(hidden=[3], tf_name='Sigmoid')

In [3]:
#net.load('../examples/speech/net_speech_500.net')

In [3]:
#dataset = open_shelve('../examples/mnist/dataset_mnist_4+9.ds')
dataset = open_shelve('../examples/rpe/dataset_rpe.ds')

In [4]:
net.fit(x=dataset['x'], y=dataset['y'], x_val=dataset['x_val'], y_val=dataset['y_val'], 
        learning_rate=0.1, n_epoch=100, req_err=0.0, batch_size=1)


--------------------------------------------------------------------
-- Network initialized.
	% problem dimension: 4
	% number of classes: 2
	% class labels: [0, 1]
	% net structure: [4, 3, 2]
	% net transfer function: Sigmoid

--------------------------------------------------------------------
-- Learning has started...
	% problem dimension: 4
	% number of training samples: 8000
	% number of validation samples: 1000
	% learning rate: 0.1
	% mini-batch size: 1
	% maximum number of epochs (t.c.): 100
	% maximum number of stable epochs (t.c.): inf
	% required accuracy (t.c.): inf
	% required error (t.c.): 0.0


epoch   on training data      on validation data       epoch time          
-------------------------------------------------------------------
 1	  0.94/0.0370		0.93/0.0383		0.7144 s
 2	  0.94/0.0318		0.93/0.0330		0.7531 s
 3	  0.94/0.0281		0.93/0.0290		0.9081 s
 4	  0.94/0.0176		0.93/0.0181		0.7683 s
 5	  1.00/0.0076		1.00/0.0078		0.7498 s
 6	  1.00/0.0037		1.00/0.0037		0.7553

In [8]:
net.learning.kw['learning_rate'] = 0.07
net.learning.learn_()


--------------------------------------------------------------------
-- Learning has started...
	% problem dimension: 784
	% number of training samples: 9847
	% number of validation samples: 1944
	% learning rate: 0.07
	% mini-batch size: 5
	% maximum number of epochs (t.c.): 500
	% maximum number of stable epochs (t.c.): inf
	% required accuracy (t.c.): 1.0
	% required error (t.c.): -inf


epoch   on training data      on validation data       epoch time          
-------------------------------------------------------------------
 1	  1.00/0.0018		0.98/0.0098		0.9778 s
 2	  1.00/0.0018		0.98/0.0095		0.9361 s
 3	  1.00/0.0018		0.98/0.0099		0.9366 s
 4	  1.00/0.0018		0.98/0.0097		0.9399 s
 5	  1.00/0.0018		0.98/0.0097		0.9315 s
 6	  1.00/0.0018		0.98/0.0099		0.9417 s
 7	  1.00/0.0018		0.98/0.0099		0.9306 s
 8	  1.00/0.0018		0.98/0.0097		0.9284 s
 9	  1.00/0.0018		0.98/0.0099		0.9227 s
 10	  1.00/0.0018		0.98/0.0098		0.9251 s
 11	  1.00/0.0018		0.98/0.0098		1.1082 s
 12	  1.00/0.0018		

KeyboardInterrupt: 

In [5]:
net.prune(req_acc=1.0, req_err=0.05, n_epoch=20, levels=(10, 7, 5, 3, 2, 1, 0))


--------------------------------------------------------------------
-- Pruning has started...
	% net initial structure: [4, 3, 2]
	% net initial number of synapses (w, b): (18, 5)
	% min required accuracy (s.c.): 1.0
	% max required error (s.c.): 0.05
	% maximum number of re-training epochs (gu.c.): 20
	% number of stable iterations (gu.c.): 10


step    trying to cut     structure           left synapses (w/b)    retrained      next level     step time      
--------------------------------------------------------------------------------------------------------------
0       0                 [4, 3, 2]           (18, 5)                None           10             None  
1       2                 [4, 3, 2]           (16, 5)                yes            10             1.026 s
2       2                 [4, 3, 2]           (14, 5)                yes            10             1.212 s
3       2                 [2, 3, 2]           (12, 5)                no             7              18.4

In [7]:
print net.w[0]
print net.w[1]

[[-5.34145254 -5.31704258  0.          0.        ]
 [ 8.73723377 -5.46579654 -0.         -0.        ]
 [ 5.11704645 -9.42493925  3.50303725  3.26011484]]
[[  8.05744705 -10.03175026  10.33757566]
 [ -8.12362451  10.01114443 -10.32462043]]


In [6]:
print 'Net structure to be dumped:', net.structure, '| Number of synapses:', net.count_synapses()
net.dump('../examples/mnist/net_mnist_4+9.net')

Net structure to be dumped: [784, 7, 2] | Number of synapses: (5502, 9)

--------------------------------------------------------------------
-- Net dumped as ../examples/mnist/net_mnist_4+9.net


In [7]:
net.v_data = net.prepare_data(x=dataset['x_val'], y=dataset['y_val'])
net.t_data = net.prepare_data(x=dataset['x'], y=dataset['y'])
test_data = net.prepare_data(x=dataset['x_test'], y=dataset['y_test'])

In [8]:
predictions = [net.predict(x)[0][0] for x, y in test_data]
cm = confusion_matrix(y_true=dataset['y_test'], y_pred=predictions, labels=net.labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
labels = [label for label in net.labels if label in dataset['y_test']]
plt.imshow(cm, aspect='auto', interpolation='none', vmin=0, vmax=1)
plt.xticks(range(len(labels)), labels)
plt.yticks(range(len(labels)), labels)
plt.grid()
plt.colorbar()
plt.show()

print accuracy_score(y_true=dataset['y_test'], y_pred=predictions)
print cm

0.442842269651
[[ 0.542       0.002       0.          0.          0.          0.          0.002
   0.028       0.          0.          0.          0.          0.004       0.01
   0.          0.004       0.264       0.014       0.          0.          0.032
   0.          0.          0.042       0.          0.          0.          0.004
   0.022       0.          0.004       0.          0.002       0.          0.002
   0.          0.002       0.          0.02        0.        ]
 [ 0.          0.466       0.          0.          0.          0.          0.
   0.          0.02        0.004       0.092       0.          0.          0.
   0.022       0.11        0.016       0.          0.048       0.          0.006
   0.018       0.          0.004       0.          0.          0.006       0.
   0.          0.          0.          0.024       0.          0.034       0.104
   0.          0.          0.026       0.          0.        ]
 [ 0.002079    0.00831601  0.65280665  0.          0.      

In [23]:
net.learning.kw['learning_rate'] = 0.001
net.learning.kw['n_epoch'] = 50
net.learning.learn_()


--------------------------------------------------------------------
-- Learning has started...
	% problem dimension: 76
	% number of training samples: 16000
	% number of validation samples: 50
	% learning rate: 0.001
	% mini-batch size: 1
	% maximum number of epochs (t.c.): 50
	% maximum number of stable epochs (t.c.): 20
	% required accuracy (t.c.): 1.0
	% required error (t.c.): -inf


epoch   on training data      on validation data       epoch time          
-------------------------------------------------------------------
 1	  0.99/0.0046		0.26/0.3610		1.7048 s
 2	  0.99/0.0046		0.26/0.3603		1.6259 s
 3	  0.99/0.0046		0.26/0.3597		1.4826 s
 4	  0.99/0.0045		0.26/0.3591		1.5106 s
 5	  0.99/0.0045		0.26/0.3585		1.5079 s
 6	  0.99/0.0045		0.26/0.3579		1.5470 s
 7	  0.99/0.0045		0.26/0.3574		1.4907 s
 8	  0.99/0.0044		0.26/0.3569		1.5318 s
 9	  0.99/0.0044		0.26/0.3564		1.5330 s
 10	  0.99/0.0044		0.26/0.3559		1.6698 s
 11	  0.99/0.0044		0.26/0.3555		1.4969 s
 12	  0.99/0.0044		0.2